# Loading of the Raw Data into DL from Snowflake

### Import needed packages

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta, date, time
import datetime
import pytz
import os

StatementMeta(, , -1, SessionStarting, , SessionStarting)

### Run the BUild Lists notebook to get our Small and Large Table Lists

In [ ]:
%run Variable_notebooks/Snowflake_BuildLists

### This code below is to attach to an Azure Storage Account

### This will need to be modified if writing to a Fabric One Lake

In [ ]:
project_name = "PROJECT NAME"
storage_account = "AZURE STORAGE ACCOUNT NAME"
container_name = "AZURE CONTAINER NAME"
 

##adding paths for Data Lake in storage account
dl_path_bronze ="BRONZE PATH"
dl_path_silver = "SILVER PATH"


## This is to save data into a spark table, remove this line if using for Fabric One Lake
spark.conf.set("myapp.dbname",dbname)
## ------------------

## Build variables for the full paths to the folders in the Data Lake
bronze_file_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/{dl_path_bronze}/{project_name}"
silver_file_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/{dl_path_silver}/{project_name}"

# Get the current date and time in EST
now_est = datetime.datetime.now(pytz.timezone('US/Eastern'))

StatementMeta(DEVPoolNick, 71, 19, Finished, Available, Finished)

In [ ]:
print(bronze_file_path)

In [ ]:
print(small_tables)

## Iterate through the small_tables list and write data to .csv file


**The files can be written to Parquet just by changing the .to_csv --> .to_parquet**

In [ ]:
for table in small_tables:
    try:
        print(f"Selecting from {table}")
        query = f'SELECT * FROM "{table}"'
        cursor.execute(query)
    
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]

        df = pd.DataFrame(rows, columns=columns)

        file_name = f"{bronze_file_path}/{table}.csv"

        df.to_csv(file_name,index=False)
        
        ## ----------------------------------
        ## To Parquet Example
        ##df.to_parquet(file_name.replace('.csv', '.parquet'), index=False) 
        ## ----------------------------------
        
        print(f"Done with {table}")
        # cursor.close()

    except Exception as e:
        print(f"An error occurred: {e}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Something went wrong: {err}")
    print("")

## Writing to Microsoft Fabric OneLake

This notebook shows a portable approach to write the DataFrame output into Microsoft Fabric OneLake by using the Microsoft Graph Drive API (works from outside Fabric notebooks). It provides:

- A short explanation and assumptions.
- Helper functions to convert DataFrames to bytes (CSV / Parquet).
- A small-file upload (simple PUT) and a chunked upload using an upload session (for files > 4 MB).

Assumptions:
- You have an Azure AD app registration with client credentials (tenant_id, client_id, client_secret).
- You know the OneLake drive id that represents the OneLake area where you want to write (we call it `oneLake_drive_id`).

Notes:
- Replace the placeholder values below with your real credentials and drive id.
- For parquet support you will need `pyarrow` or `fastparquet` installed. For authentication we use `msal`.
- This example writes CSV/parquet bytes to OneLake via Microsoft Graph. If you're running inside a Fabric notebook there may be simpler environment-native APIs; tell me if you want that instead.


In [ ]:
### THIS CODE WAS BUILT BY COPILOT AND HAS NOT BEEN USED IN DEV OR TESTING USE AS EXAMPLE ONLY

# OneLake upload helper functions (using Microsoft Graph)
# Install msal if you don't have it: pip install msal
import io
import msal
import requests
import os

# --- Configuration (replace with your values) ---
tenant_id = os.environ.get('AZURE_TENANT_ID', '<YOUR_TENANT_ID>')
client_id = os.environ.get('AZURE_CLIENT_ID', '<YOUR_CLIENT_ID>')
client_secret = os.environ.get('AZURE_CLIENT_SECRET', '<YOUR_CLIENT_SECRET>')
oneLake_drive_id = os.environ.get('ONELAKE_DRIVE_ID', '<YOUR_ONELAKE_DRIVE_ID>')

# Scopes for app-only Graph access
scope = ["https://graph.microsoft.com/.default"]

# Acquire token (client credentials)
app = msal.ConfidentialClientApplication(client_id, authority=f"https://login.microsoftonline.com/{tenant_id}", client_credential=client_secret)
resp = app.acquire_token_for_client(scopes=scope)
if 'access_token' not in resp:
    raise Exception(f"Could not obtain access token: {resp}")

ACCESS_TOKEN = resp['access_token']
HEADERS = {"Authorization": f"Bearer {ACCESS_TOKEN}"}


def df_to_bytes(df, fmt='csv'):
    """Convert DataFrame to bytes in CSV or parquet format."""
    if fmt == 'csv':
        bio = io.BytesIO()
        df.to_csv(bio, index=False)
        bio.seek(0)
        return bio.read()
    elif fmt == 'parquet':
        bio = io.BytesIO()
        df.to_parquet(bio, index=False)
        bio.seek(0)
        return bio.read()
    else:
        raise ValueError('Unsupported format')


def upload_small_file(drive_id, path_in_drive, data_bytes, content_type='text/csv'):
    """Upload small files (<4MB) by simple PUT to the Graph item endpoint.
    path_in_drive should be like '/path/to/file.csv' relative to the drive root.
    """
    url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root:{path_in_drive}:/content"
    headers = HEADERS.copy()
    headers['Content-Type'] = content_type
    r = requests.put(url, headers=headers, data=data_bytes)
    r.raise_for_status()
    return r.json()


def create_upload_session(drive_id, path_in_drive):
    url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root:{path_in_drive}:/createUploadSession"
    r = requests.post(url, headers=HEADERS, json={"item": {"@microsoft.graph.conflictBehavior": "replace"}})
    r.raise_for_status()
    return r.json()


def upload_large_file_via_session(upload_url, data_bytes, chunk_size=5 * 1024 * 1024):
    """Upload bytes via an upload session. chunk_size defaults to 5MB (Graph minimum is 5MB for chunked uploads)."""
    total = len(data_bytes)
    start = 0
    while start < total:
        end = min(start + chunk_size, total) - 1
        chunk = data_bytes[start:end+1]
        headers = {
            'Content-Length': str(len(chunk)),
            'Content-Range': f'bytes {start}-{end}/{total}'
        }
        r = requests.put(upload_url, headers=headers, data=chunk)
        if not (200 <= r.status_code < 300 or r.status_code == 201):
            raise Exception(f"Upload failed: {r.status_code} {r.text}")
        start = end + 1
    return r.json()

# Example usage inside the loop for small_tables (replace call to df.to_csv with upload_small_file)
# For example in the earlier loop after df creation you can do:
# bytes_data = df_to_bytes(df, fmt='csv')
# remote_path = f"/Bronze/{project_name}/{table}.csv"
# upload_small_file(oneLake_drive_id, remote_path, bytes_data, content_type='text/csv')

print('OneLake helper functions loaded. Replace placeholders and call the upload functions where appropriate.')


### Iterate through large_tables and write data to .csv file


***Can be written to Parquet***

***This Code performs "Chunking" that was needed for a specific project. If you do not need this then just write contents to .csv or .parquet and remove the chunking code***

In [ ]:
for table in largeTables:
    import pandas as pd

    #step 1 get id col from control table
    #step 2 Look at table if id col exist if not drop from the table list and do not ingest 
    #step 3 ingest tables that match into dw

    # Define chunk size
    chunk_size = 400000

    # Get total record count from Snowflake
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    record_count = cursor.fetchone()[0]

    # Create chunks for processing
    first_last_ids = [(first_id, last_id) for first_id, last_id in zip(range(1, record_count+1, chunk_size), range(chunk_size, record_count+1, chunk_size))]

    for i, (first_id, last_id) in enumerate(first_last_ids, start=1):
        print(f'Chunk {i}: First ID = {first_id}, Last ID = {last_id}')
    
        # Query to fetch the chunk of data
        sql_query = f"SELECT * FROM "{table}" WHERE Id BETWEEN {first_id} AND {last_id}"
        print(sql_query)

        # Execute the query
        cursor.execute(sql_query)
    
        # Fetch data and column names
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]

        # Convert to DataFrame
        df = pd.DataFrame(rows, columns=columns)
    
        # Save to CSV
        filename = f'{bronze_file_path}/{table_name}-{i:02}.csv'
        df.to_csv(filename, index=False)
        print(f'Data from {table_name} saved in {filename}')
    
    


### Close the connector from the build_lists Table

In [ ]:
cursor.close()
conn.close()

# Modifications for Fabric to consider

- You may not want to break out the connection between notebooks like we did, you can put the cursor code into this one notebook and execute directly. By doing multiple notebooks you can make your conections to Snowflake More Dynamic
- Code to write files will need to be modified so that you can write to Fabric Onelake. CODE ABOVE CREATED BY AI FOR EXAMPLE USE ONLY
